In [151]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [152]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [153]:
pip install torchsummary


Note: you may need to restart the kernel to use updated packages.


In [154]:
from statistics import mean
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
from torchvision import models

In [155]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    device='cpu'
    print('CUDA is not available.  Training on CPU ...')
else:
    device='cuda'
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [156]:
## read raw data 
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_test= pd.read_csv('../input/digit-recognizer/test.csv')
labels = torch.tensor(df_train['label'].values.astype(np.float32)).long()

In [157]:
## put it into a data set
df_train= df_train.drop(['label'], axis=1)

#normalize to an image
train_data = torch.tensor(df_train.values.reshape((-1,28,28)).astype('float32'))/255
test_data = df_test.values.reshape((-1,28,28))

training_dataset = torch.utils.data.TensorDataset(train_data, labels)

train_size = int(0.8 *len(training_dataset))
validation_size = len(training_dataset) - train_size

train_dataset, validation_dataset = torch.utils.data.random_split(training_dataset, [train_size, validation_size])

batch_size = 1

dataloaders = dict()
dataloaders['train'] = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
dataloaders['validation'] = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size)


In [158]:
import torch.nn as nn
from torch.nn import functional as F

class digitnet(nn.Module):
    
    input_size = [28, 28]
    output_size = 10
    input_channels = 1
    channels_conv1 = 9
    channels_conv2 = 18
    kernel_conv1 = [3, 3]
    kernel_conv2 = [3, 3]
    pool_conv1 = [2, 2]
    pool_conv2 = [1, 2]
    fcl1_size = 50
    
    def __init__(self):
        super(digitnet, self).__init__()
        
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(self.input_channels, self.channels_conv1, self.kernel_conv1)
        self.conv2 = nn.Conv2d(self.channels_conv1, self.channels_conv2, self.kernel_conv2)
        
        # Calculate the convolutional layers output size (stride = 1)
        c1 = np.array(self.input_size) - self.kernel_conv1 + 1
        p1 = c1 // self.pool_conv1
        c2 = p1 - self.kernel_conv2 + 1
        p2 = c2 // self.pool_conv2
        self.conv_out_size = int(p2[0] * p2[1] * self.channels_conv2)
        
        # Define the fully connected layers
        self.fcl1 = nn.Linear(self.conv_out_size, self.fcl1_size)
        self.fcl2 = nn.Linear(self.fcl1_size, self.output_size)
        
    def forward(self, x):
        
        # reshape x and input size of network
        x= x.unsqueeze(1)
        # Apply convolution 1 and pooling
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, self.pool_conv1)
        
        # Apply convolution 2 and pooling
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, self.pool_conv2)
        
        # Reshape x to one dimmension to use as input for the fully connected layers
        x = x.view(-1, self.conv_out_size)
        
        # Fully connected layers
        x = self.fcl1(x)
        x = F.relu(x)
        x = self.fcl2(x)
        
        ## Last layer as per problem
        x = F.log_softmax(x)
        return x
        

In [159]:
model = digitnet().to(device)
print(model)

digitnet(
  (conv1): Conv2d(1, 9, kernel_size=[3, 3], stride=(1, 1))
  (conv2): Conv2d(9, 18, kernel_size=[3, 3], stride=(1, 1))
  (fcl1): Linear(in_features=990, out_features=50, bias=True)
  (fcl2): Linear(in_features=50, out_features=10, bias=True)
)


In [160]:
learning_rate = 0.25
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [161]:
from torch.autograd import Variable

train_losses = []
validation_losses = []
test_accuracies=[]


epochs = 20
for epoch in range(epochs):
    model.train()
    batch_train_losses =[]
    batch_val_losses = []
    
    for i, (observations, labels) in enumerate(dataloaders['train']):
        
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(observations.to(device))
        
        # Backward pass
        loss= criterion(outputs, labels.to(device))
        loss.backward()
        
        
        # Optimize
        optimizer.step()
        
        # Accumulate losses
        batch_train_losses.append(loss.cpu().detach().numpy())
        
        
    # Testing on validation data
    model.eval()
    
    correct = 0
    total = 0
    
    
    for i,(observations, labels) in enumerate(dataloaders['validation']):
        
        # Forward pass
        outputs = model(observations.to(device))
        
        # Backward pass
        loss = criterion(outputs, labels.to(device))
        
        # Accumulate losses
        batch_val_losses.append(loss.cpu().detach().numpy())
        
        #Calculate accuracy
        _,predicted = torch.max(outputs.data, 1)
        correct += (predicted.cpu() == labels).sum().item()
        total += labels.size(0)
        
    #Accumulate train and validation loss/accuracy for the epoch
    train_losses.append(np.mean(batch_train_losses))
    validation_losses.append(np.mean(batch_val_losses))
    
    accuracy = 100 * correct/total
    test_accuracies.append(accuracy)
    
    
    print('Epoch {} - Training Loss : {:.4f}, Testing Loss: {:.4f}, Test accuracy: {:.2f}% '\
          .format(epoch+1, np.mean(batch_train_losses), np.mean(batch_val_losses), accuracy))
        
    
        
plt.plot(train_losses)
plt.show()
plt.plot(validation_losses)
        
        

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!